In [1]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
# import some relevant packages:
import gym, torch, numpy as np, torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import tianshou as ts

import sys
sys.path.append("../src/")
from util import LOGGER_FOLDER, MODEL_FOLDER

In [2]:
# Define some hyper-parameters:

task = 'CartPole-v0'
lr, epoch, batch_size = 1e-3, 10, 64
train_num, test_num = 10, 100
gamma, n_step, target_freq = 0.9, 3, 320
buffer_size = 20000
eps_train, eps_test = 0.1, 0.05
step_per_epoch, step_per_collect = 10000, 10
logger = ts.utils.TensorboardLogger(SummaryWriter(LOGGER_FOLDER / 'dqn'))  # TensorBoard is supported!
# For other loggers: https://tianshou.readthedocs.io/en/master/tutorials/logger.html

2022-02-20 10:12:10.118909: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/vincent/.mujoco/mujoco210/bin/:/home/vincent/.mujoco/mujoco200/bin/:/usr/lib/nvidia:/home/vincent/.mujoco/mujoco210/bin/:/home/vincent/.mujoco/mujoco200/bin/:/usr/lib/nvidia
2022-02-20 10:12:10.118933: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# Make environments:

# you can also try with SubprocVectorEnv
train_envs = ts.env.DummyVectorEnv([lambda: gym.make(task) for _ in range(train_num)])
test_envs = ts.env.DummyVectorEnv([lambda: gym.make(task) for _ in range(test_num)])

In [4]:
# Define the network
from tianshou.utils.net.common import Net
# you can define other net by following the API:
# https://tianshou.readthedocs.io/en/master/tutorials/dqn.html#build-the-network
env = gym.make(task)
state_shape = env.observation_space.shape or env.observation_space.n
action_shape = env.action_space.shape or env.action_space.n
net = Net(state_shape=state_shape, action_shape=action_shape, hidden_sizes=[128, 128, 128])
optim = torch.optim.Adam(net.parameters(), lr=lr)

In [5]:
# Setup policy and collectors
policy = ts.policy.DQNPolicy(net, 
                             optim, 
                             gamma, 
                             n_step, 
                             target_update_freq=target_freq)
train_collector = ts.data.Collector(policy, 
                                    train_envs, 
                                    ts.data.VectorReplayBuffer(buffer_size, train_num), 
                                    exploration_noise=True)
test_collector = ts.data.Collector(policy, 
                                   test_envs, 
                                   exploration_noise=True)  # because DQN uses epsilon-greedy method

In [6]:
# train it
result = ts.trainer.offpolicy_trainer(
    policy, train_collector, test_collector, epoch, step_per_epoch, step_per_collect,
    test_num, batch_size, update_per_step=1 / step_per_collect,
    train_fn=lambda epoch, env_step: policy.set_eps(eps_train),
    test_fn=lambda epoch, env_step: policy.set_eps(eps_test),
    stop_fn=lambda mean_rewards: mean_rewards >= env.spec.reward_threshold,
    logger=logger)
print(f'Finished training! Use {result["duration"]}')

Epoch #1: 10001it [00:03, 3322.40it/s, env_step=10000, len=140, loss=0.349, n/ep=0, n/st=10, rew=140.00]                                                      


Epoch #1: test_reward: 173.400000 ± 28.834701, best_reward: 173.400000 ± 28.834701 in #1


Epoch #2:  43%|#####################4                            | 4290/10000 [00:02<00:03, 1569.75it/s, env_step=14290, len=200, n/ep=1, n/st=10, rew=200.00]

Finished training! Use 6.14s


In [8]:
# Save / load the trained policy (it's exactly the same as PyTorch nn.module):
torch.save(policy.state_dict(), MODEL_FOLDER / 'dqn.pth')
policy.load_state_dict(torch.load(MODEL_FOLDER / 'dqn.pth'))

<All keys matched successfully>

In [9]:
# Watch the performance with 35 FPS
policy.eval()
policy.set_eps(eps_test)
collector = ts.data.Collector(policy, env, exploration_noise=True)
collector.collect(n_episode=1, render=1 / 35)

/home/vincent/.local/lib/python3.8/site-packages/tianshou/data/collector.py:61: UserWarning: Single environment detected, wrap to DummyVectorEnv.
  warnings.warn("Single environment detected, wrap to DummyVectorEnv.")


NoSuchDisplayException: Cannot connect to "None"